In [1]:
import os 
packages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)
os.environ["PYSPARK_PYTHON"] = '/usr/bin/python3'



from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example-pyspark-read-and-write").master("spark://spark-master:7077").config('spark.cores.max','2').config("spark.executor.memory", "512m").getOrCreate()

In [2]:
# HDFS_HOST="hdfs://namenode:9000/"
# # Create data
# data = [('First', 1), ('Second', 2), ('Third', 3), ('Fourth', 4), ('Fifth', 5)]
# df = sparkSession.createDataFrame(data)

# # Write into HDFS
# df.write.csv(HDFS_HOST+"test/example.csv")
# # để kiểm tra vào shell của name node 
# # dùng lệnh hdfs dfs -ls /
# # sẽ hiển thị thông tin file vừa ghi

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([   
        StructField("time", StringType(), True),
        StructField("text", StringType(), True),
        StructField("retweet_count", DoubleType(), True),
        StructField("location", StringType(), True),
        StructField("favorite_count", DoubleType(), True),
        StructField("user_id", StringType(), True),
        StructField("place", StringType(), True),
        StructField("user_followers_count", StringType(), True),
        StructField("sentiment", StringType(), True)
    
])

In [4]:
bidenDF = spark \
    .readStream \
    .schema(schema) \
    .csv("hdfs://namenode:9000/data/biden.csv/")

trumpDF = spark \
    .readStream \
    .schema(schema) \
    .csv("hdfs://namenode:9000/data/trump.csv/")

bidenDF = bidenDF.withColumn("value", to_json(struct([bidenDF[x] for x in bidenDF.columns])))
trumpDF = trumpDF.withColumn("value", to_json(struct([trumpDF[x] for x in trumpDF.columns])))


In [5]:
bidenDF \
  .selectExpr("CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092")\
  .option("checkpointLocation", "/tmp/biden-visualize/checkpoint")\
  .option("topic", "biden_visualize") \
  .start()


trumpDF \
  .selectExpr("CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092")\
  .option("checkpointLocation", "/tmp/trump-visualize/checkpoint")\
  .option("topic", "trump_visualize") \
  .start()